In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import Input, Output, MLClient, command
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.parallel import parallel_run_function, RunFunction

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

#TODO: your compute name.
cpu_compute_target = "xxx"
print(ml_client.compute.get(cpu_compute_target))

In [ ]:
data_prep_component = command(
    name="prep_index",
    display_name="Create partition index for training",
    description="Create partition index for deltalake data",
    inputs=dict(
        data=Input(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT
        ),
    ),
    outputs=dict(
        index_data=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT
        ),
    ),
    code="./src",
    command="""python index_prep.py \
            --data_input ${{inputs.data}} --index_output ${{outputs.index_data}}\
            """,
    environment=Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        conda_file="./job-env/conda_dependencies.yml",
    ),
    compute=cpu_compute_target,
)

In [ ]:

prs_delta = parallel_run_function(
    name="prs_delta",
    display_name="prs example to read delta",
    description="prs example to read delta lake data",
    inputs=dict(
        index_folder=Input(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT
        ),
        data=Input(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT
        ),
    ),
    input_data="${{inputs.index_folder}}",
    instance_count=1,
    max_concurrency_per_instance=1,
    mini_batch_error_threshold=1,
    mini_batch_size="1",
    retry_settings=dict(max_retries=1, timeout=3600),
    compute = cpu_compute_target,
    logging_level="DEBUG",
    task=RunFunction(
        code="./src",
        environment=Environment(
            image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
            conda_file="./job-env/conda_dependencies.yml",
        ),
        entry_script='train_parallel.py',
        program_arguments="--data_input ${{inputs.data}}"  # Passthrough input data folder into script.
        ,
    ),
)

In [ ]:
#TODO: change to your deltalake uri_folder
data_folder=Input(
    type=AssetTypes.URI_FOLDER,
    path= 'azureml:deltalakefolder:1',
    mode=InputOutputModes.RO_MOUNT
)

In [ ]:
@pipeline(
    display_name="parallel job for deltalake data",
)
def partition_job_in_pipeline(pipeline_input):
    data_prep_step = data_prep_component(data = pipeline_input)
    parallel_train = prs_delta(data = pipeline_input, index_folder = data_prep_step.outputs.index_data)

In [ ]:
my_job = partition_job_in_pipeline(data_folder)

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    my_job,
    experiment_name="deltalake-parallel-job",
)
pipeline_job